# DE Africa Coastlines raster generation <img align="right" src="https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/raw/main/Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

This code conducts raster generation for DE Africa Coastlines:

* Load stack of all available Landsat 5, 7 and 8 satellite imagery for a location using [ODC Virtual Products](https://docs.dea.ga.gov.au/notebooks/Frequently_used_code/Virtual_products.html)
* Convert each satellite image into a remote sensing water index (MNDWI)
* For each satellite image, model ocean tides into a 5 x 5 km grid based on exact time of image acquisition
* Interpolate tide heights into spatial extent of image stack
* Mask out high and low tide pixels by removing all observations acquired outside of 50 percent of the observed tidal range centered over mean sea level
* Combine tidally-masked data into annual median composites representing the most representative position of the coastline at approximately mean sea level each year

This is an interactive version of the code intended for prototyping; to run this analysis at scale, use the [command line tools](DEAfricaCoastlines_generation_CLI.ipynb).

---

## Getting started
Set working directory to top level of repo to ensure links work correctly:

In [1]:
cd ..

/g/data/dea-coastlines/deafrica-coastlines


### Load packages

First we import the required Python packages, then we connect to the database, and load the catalog of virtual products.

In [2]:
pip install -r requirements.in --quiet

You should consider upgrading via the '/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%matplotlib inline
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import os
import sys
import dask
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from functools import partial
from datacube.utils.geometry import Geometry
from dea_tools.dask import create_local_dask_cluster

# Load DE Africa Coastlines code
import coastlines.raster

# Connect to datacube
import datacube

dc = datacube.Datacube(app="DEAfricaCoastlines")

# Create local dask client for parallelisation
client = create_local_dask_cluster(return_client=True)

Client Scheduler: tcp://127.0.0.1:33605 Dashboard: /user/robbi.bishoptaylor@ga.gov.au/proxy/8787/status,Cluster Workers: 1 Cores: 31 Memory: 254.70 GB


## Setup


### Set analysis parameters

In [4]:
study_area = 696
raster_version = 'cli_update'
start_year = 2000
end_year = 2020

# Load analysis params from config file
config = coastlines.raster.load_config(
    config_path='configs/deafrica_coastlines_config.yaml')

### Load supplementary data

In [5]:
# Tide points are used to model tides across the extent of the satellite data
points_gdf = gpd.read_file(config['Input files']['coastal_points_path'])

# Albers grid cells used to process the analysis
gridcell_gdf = (
    gpd.read_file(config['Input files']['coastal_grid_path']).to_crs(
        epsg=4326).set_index('id'))
gridcell_gdf.index = gridcell_gdf.index.astype(int).astype(str)
gridcell_gdf = gridcell_gdf.loc[[str(study_area)]]

## Loading data
### Create spatiotemporal query
This establishes the spatial and temporal extent used to search for Landsat satellite data.


In [6]:
# Create query based on analysis gridcell extent
geopoly = Geometry(gridcell_gdf.iloc[0].geometry, crs=gridcell_gdf.crs)
query = {
    'geopolygon': geopoly.buffer(0.05),
    'time': (str(start_year - 1), str(end_year + 1)),  # 1999, 2021
    'dask_chunks': {
        'time': 1,
        'x': 3000,
        'y': 3000
    }
}

### Load satellite data as MNDWI
This step loads satellite data from Landsat 5, 7 and 8, and returns the data as a cloud-masked array converted to the Modified Normalised Difference Water Index (MNDWI).
For Digital Earth Australia Coastlines, this is achieved using [ODC Virtual Products](https://docs.dea.ga.gov.au/notebooks/Frequently_used_code/Virtual_products.html).

In [7]:
# Load MNDWI virtual product
product_name = "ls_sr_st"
ds = coastlines.raster.load_water_index(
    dc,
    query,
    yaml_path=config["Virtual product"]["virtual_product_path"],
    product_name=config["Virtual product"]["virtual_product_name"],
    mask_terrain_shadow=False,
)
ds

<xarray.Dataset>
Dimensions:      (time: 1045, y: 1316, x: 1003)
Coordinates:
  * time         (time) datetime64[ns] 1999-01-04T08:45:23.581013 ... 2021-12...
  * y            (y) float64 -1.295e+06 -1.295e+06 ... -1.335e+06 -1.335e+06
  * x            (x) float64 3.61e+05 3.61e+05 3.61e+05 ... 3.91e+05 3.91e+05
    spatial_ref  int32 32633
Data variables:
    mndwi        (time, y, x) float32 dask.array<chunksize=(1, 1316, 1003), meta=np.ndarray>
    ndwi         (time, y, x) float32 dask.array<chunksize=(1, 1316, 1003), meta=np.ndarray>
Attributes:
    crs:           epsg:32633
    grid_mapping:  spatial_ref

## Tidal modelling
### Model tides at point locations
Model tides at each point in a provided `geopandas.GeoDataFrame` based on all timesteps observed by Landsat. This returns a new `geopandas.GeoDataFrame` with a "time" index 
(matching every time step in our Landsat data), and a "tide_m" column giving the 
tide heights at each point location at that time.

In [11]:
tidepoints_gdf = coastlines.raster.model_tide_points(ds, points_gdf, directory='/var/share')
tidepoints_gdf.plot()

FileNotFoundError: ['/var/share/fes2014/ocean_tide/2n2.nc', '/var/share/fes2014/ocean_tide/eps2.nc', '/var/share/fes2014/ocean_tide/j1.nc', '/var/share/fes2014/ocean_tide/k1.nc', '/var/share/fes2014/ocean_tide/k2.nc', '/var/share/fes2014/ocean_tide/l2.nc', '/var/share/fes2014/ocean_tide/la2.nc', '/var/share/fes2014/ocean_tide/m2.nc', '/var/share/fes2014/ocean_tide/m3.nc', '/var/share/fes2014/ocean_tide/m4.nc', '/var/share/fes2014/ocean_tide/m6.nc', '/var/share/fes2014/ocean_tide/m8.nc', '/var/share/fes2014/ocean_tide/mf.nc', '/var/share/fes2014/ocean_tide/mks2.nc', '/var/share/fes2014/ocean_tide/mm.nc', '/var/share/fes2014/ocean_tide/mn4.nc', '/var/share/fes2014/ocean_tide/ms4.nc', '/var/share/fes2014/ocean_tide/msf.nc', '/var/share/fes2014/ocean_tide/msqm.nc', '/var/share/fes2014/ocean_tide/mtm.nc', '/var/share/fes2014/ocean_tide/mu2.nc', '/var/share/fes2014/ocean_tide/n2.nc', '/var/share/fes2014/ocean_tide/n4.nc', '/var/share/fes2014/ocean_tide/nu2.nc', '/var/share/fes2014/ocean_tide

In [10]:
tidepoints_gdf

,tide_m,geometry
time,,
1999-01-04 08:45:23.581013+00:00,-0.520841,POINT (363425.391 -1290265.874)
1999-01-20 08:45:19.978056+00:00,-0.437340,POINT (363425.391 -1290265.874)
1999-02-05 08:45:27.257019+00:00,-0.061844,POINT (363425.391 -1290265.874)
1999-02-12 08:51:43.040075+00:00,-0.139849,POINT (363425.391 -1290265.874)
1999-04-10 08:45:01.171081+00:00,0.150013,POINT (363425.391 -1290265.874)
...,...,...
2021-12-10 09:06:39.834008+00:00,0.236414,POINT (358006.979 -1339001.197)
2021-12-18 09:06:39.209734+00:00,-0.482638,POINT (358006.979 -1339001.197)
2021-12-25 09:12:47.572883+00:00,0.049211,POINT (358006.979 -1339001.197)


### Interpolate tides into each satellite timestep
For each satellite timestep, spatially interpolate our modelled tide height points into the spatial extent of our satellite image, and add this new data as a new variable in our satellite dataset. This allows each satellite pixel to be analysed and filtered/masked based on the tide height at the exact moment of satellite image acquisition. 

In [ ]:
# Interpolate tides for each timestep in `ds`
ds["tide_m"] = coastlines.raster.multiprocess_apply(
    ds=ds,
    dim="time",
    func=partial(coastlines.raster.interpolate_tide,
                 tidepoints_gdf=tidepoints_gdf))

Plot example interpolated tide surface for a single timestep:

In [ ]:
import matplotlib.pyplot as plt

# Plot
ds_i = ds['tide_m'].isel(time=-1).compute()
ds_i.plot.imshow(robust=True,
                 cmap='viridis',
                 size=12,
                 vmin=ds_i.min().item(),
                 vmax=ds_i.max().item())
tidepoints_gdf.loc[str(ds_i.time.values)[0:10]].plot(ax=plt.gca(),
                                                     column='tide_m',
                                                     cmap='viridis',
                                                     markersize=100,
                                                     edgecolor='black',
                                                     vmin=ds_i.min().item(),
                                                     vmax=ds_i.max().item())
gridcell_gdf.to_crs(tidepoints_gdf.crs).plot(ax=plt.gca(),
                                             facecolor='none',
                                             edgecolor='black');

### Calculate per-pixel tide cutoffs
Based on the entire time-series of tide heights, compute the max and min satellite-observed tide height for each pixel, then calculate tide cutoffs used to restrict our data to satellite observations centred over mid-tide (0 m Above Mean Sea Level).

In [ ]:
# Determine tide cutoff
tide_cutoff_min, tide_cutoff_max = coastlines.raster.tide_cutoffs(ds, tidepoints_gdf)

## Generate yearly composites
Export tidally-masked MNDWI median composites for each year, and three-yearly composites used to gapfill poor data coverage areas.

In [ ]:
# If output folder doesn't exist, create it
output_dir = f'data/interim/raster/{raster_version}/{study_area}_{raster_version}'
os.makedirs(output_dir, exist_ok=True)

# Iterate through each year and export annual and 3-year gapfill composites
coastlines.raster.export_annual_gapfill(ds, output_dir, tide_cutoff_min,
                                        tide_cutoff_max)

### Close Dask client

In [ ]:
client.close()

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** For assistance with any of the Python code or Jupyter Notebooks in this repository, please post a [Github issue](https://github.com/GeoscienceAustralia/DEACoastLines/issues/new).

**Last modified:** May 2022